In [35]:
import numpy as np
from numpy import array
import pickle as plk
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Bidirectional
from keras.layers import Dropout
from keras.layers import LSTM, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
import os

In [4]:
# df = pd.read_excel('/Users/luyin/Desktop/project/Q&A.xlsx',header = 0)
df = pd.read_excel(os.getcwd()+'/Q&A_Database_new.xlsx','QA', skiprows=3)

In [5]:
l = df['Analyst name'].unique() # 79 unique analyst
dic = {} #create dictionary for questions

for name in l:
    list_ = list(df.loc[df['Analyst name']  == name]['Question'])
    dic[name] = list_

In [6]:
import spacy
import re
import string
# Load English tokenizer, tagger, parser, NER and word vectors

tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

def tokenize(sent):
#   sent = re.sub('[^A-Za-z&]', ' ', sent) # replace non-letter with space
#   sent = re.sub(r'\b[a-zA-Z]\b', '', sent) #remove single letter 
    sent = re.sub('^[0-9]+', '', sent)
    tokens = tokenizer(sent)
    return [(token.text.lower()) for token in tokens if (token.text.lower() not in punctuations and token.is_alpha and token.text.lower() not in {'\xa0', ' '," "})]

# tokens = tokenize(" going to hit them one way or another strong dollar did seem to have a huge impact and y ou, what are you doing? I'm 's what do you and me think or like apples and Apple is looking at buying and bought U.K. startup for $1 billion. '\n' another sentence")
# for token in tokens:
#     print (token)

In [7]:
def tokenize_analyst_q(name):
    all_tokens = []
    for q in dic[name]:
        tokens = tokenize(q)
        all_tokens += tokens
    return all_tokens

In [8]:
tokens = tokenize_analyst_q('Glenn Schorr')

In [9]:
len(tokens)

12657

In [10]:
def build_vocab(input_):
    vocab = sorted(set(input_))
    vocab_to_int = dict((c, i) for i, c in enumerate(vocab))
    int_to_vocab = dict((i, c) for i, c in enumerate(vocab))
    n_total = len(input_)
    n_vocab = len(vocab)
    print ("Total Words: {}".format(n_total))
    print ("Total Vocab: {}".format(n_vocab))
    return n_total, n_vocab

In [11]:
# organize into sequences of tokens
def build_sequence(length, input_):
    sequences = list()
    for l in range(length, len(input_)):
        seq = input_[l-length:l]
        line = ' '.join(seq)
        sequences.append(line)
    print('Total Sequences: {}' .format(len(sequences)))
    return sequences

In [12]:
n_total, n_vocab = build_vocab(tokens)
sequences = build_sequence(20, tokens)

Total Words: 12657
Total Vocab: 1658
Total Sequences: 12637


In [13]:
def save_file(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

out_filename = 'Glenn_Schorr_sequences.txt'
save_file(sequences, out_filename)


In [15]:
# load doc into memory
def load_file(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
 
in_filename = 'Glenn_Schorr_sequences.txt'
doc = load_file(in_filename)
lines = doc.split('\n')

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines) #fit on texts
sequences = tokenizer.texts_to_sequences(lines)
vocab_size = len(tokenizer.word_index) + 1
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [22]:
X

array([[   1,  263,    6, ...,    2,   31,   21],
       [ 263,    6,  237, ...,   31,   21,   59],
       [   6,  237,   37, ...,   21,   59,  238],
       ...,
       [ 108,    4,   82, ...,   51, 1658,  329],
       [   4,   82,    4, ..., 1658,  329,    4],
       [  82,    4,  237, ...,  329,    4,   18]])

In [24]:
words_to_load = 50000
PAD_IDX = 0
UNK_IDX = 1
import numpy as np
# reserve the 1st 2nd token for padding and <UNK> respectively
with open('/Users/cyian/Desktop/NYU/FALL2018/DS-GA1001_NLP/hws/HW2/wiki-news-300d-1M.vec') as f:
    loaded_embeddings_ft = np.zeros((words_to_load+2, 300))
    words_ft = {}
    idx2words_ft = {}
    ordered_words_ft = []
    ordered_words_ft.extend(['<pad>', '<unk>'])
    loaded_embeddings_ft[0,:] = np.zeros(300)
    loaded_embeddings_ft[1,:] = np.random.normal(size = 300)
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings_ft[i+2, :] = np.asarray(s[1:])
        words_ft[s[0]] = i+2
        idx2words_ft[i+2] = s[0]
        ordered_words_ft.append(s[0])
    words_ft['<pad>'] = 0
    words_ft['<unk>'] = 1
    idx2words_ft[0] = '<pad>'
    idx2words_ft[1] = '<unk>'

In [26]:
def word_to_id(word_list):
    return [words_ft[x] if x in ordered_words_ft else UNK_IDX for x in word_list.split()]
def sent_to_id(sent_list):
    return [word_to_id(x) for x in sent_list]

In [25]:
lines

['the performance in equities was great and you mentioned it pretty much across the board do you think there any',
 'performance in equities was great and you mentioned it pretty much across the board do you think there any seasonality',
 'in equities was great and you mentioned it pretty much across the board do you think there any seasonality any',
 'equities was great and you mentioned it pretty much across the board do you think there any seasonality any one',
 'was great and you mentioned it pretty much across the board do you think there any seasonality any one time',
 'great and you mentioned it pretty much across the board do you think there any seasonality any one time events',
 'and you mentioned it pretty much across the board do you think there any seasonality any one time events block',
 'you mentioned it pretty much across the board do you think there any seasonality any one time events block trades',
 'mentioned it pretty much across the board do you think there any seas

In [55]:
len_list = []
for x in lines:
    len_list.append(len(x.split()))

In [56]:
sorted(len_list)[-1]

20

In [57]:
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 20

In [65]:
# integer encode sequences of words
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(lines) #fit on texts
# sequences = tokenizer.texts_to_sequences(lines)
# vocab_size = len(tokenizer.word_index) + 1
MAX_NUM_WORDS = words_to_load
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
X, y = data[:,:-1], data[:,-1]
vocab_size = len(words_ft)
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
 
# define model
model = Sequential()
embedding_layer = Embedding(vocab_size,
                            EMBEDDING_DIM,
                            weights=[loaded_embeddings_ft],
                            input_length=MAX_SEQUENCE_LENGTH-1,
                            trainable=False)
model.add(embedding_layer)
model.add(Bidirectional(GRU(100, return_sequences=True)))
model.add(Bidirectional(GRU(100)))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

#import the checkpoint to save current model
filepath="GRU_0.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# compile model

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# fit the model
model.fit(X, y, batch_size=200, epochs=100, callbacks=callbacks_list)

# categorical_crossentropy
 
# save the model to file
model.save('model.h5')
# save the tokenizer
plk.dump(tokenizer, open('tokenizer.pkl', 'wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 19, 300)           15000600  
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 19, 200)           240600    
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 200)               180600    
_________________________________________________________________
dense_17 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_18 (Dense)             (None, 50002)             5050202   
Total params: 20,492,102
Trainable params: 5,491,502
Non-trainable params: 15,000,600
_________________________________________________________________
Epoch 1/100
12637/12637 [==============================] - 48s 4ms/step - loss: 7.4284 - acc: 0.0494

Epoch 00001: loss im

12637/12637 [==============================] - 41s 3ms/step - loss: 2.2386 - acc: 0.4609

Epoch 00039: loss improved from 2.36079 to 2.23857, saving model to LSTM_basline.hdf5
Epoch 40/100
12637/12637 [==============================] - 40s 3ms/step - loss: 2.1035 - acc: 0.4958

Epoch 00040: loss improved from 2.23857 to 2.10347, saving model to LSTM_basline.hdf5
Epoch 41/100
12637/12637 [==============================] - 40s 3ms/step - loss: 2.0063 - acc: 0.5118

Epoch 00041: loss improved from 2.10347 to 2.00631, saving model to LSTM_basline.hdf5
Epoch 42/100
12637/12637 [==============================] - 40s 3ms/step - loss: 1.8799 - acc: 0.5425

Epoch 00042: loss improved from 2.00631 to 1.87989, saving model to LSTM_basline.hdf5
Epoch 43/100
12637/12637 [==============================] - 41s 3ms/step - loss: 1.7701 - acc: 0.5739

Epoch 00043: loss improved from 1.87989 to 1.77014, saving model to LSTM_basline.hdf5
Epoch 44/100
12637/12637 [==============================] - 42s 3ms/

12637/12637 [==============================] - 133s 10ms/step - loss: 0.0460 - acc: 0.9973

Epoch 00082: loss did not improve from 0.04590
Epoch 83/100
12637/12637 [==============================] - 117s 9ms/step - loss: 0.0407 - acc: 0.9984

Epoch 00083: loss improved from 0.04590 to 0.04075, saving model to LSTM_basline.hdf5
Epoch 84/100
12637/12637 [==============================] - 115s 9ms/step - loss: 0.0345 - acc: 0.9990

Epoch 00084: loss improved from 0.04075 to 0.03447, saving model to LSTM_basline.hdf5
Epoch 85/100
12637/12637 [==============================] - 124s 10ms/step - loss: 0.0309 - acc: 0.9992

Epoch 00085: loss improved from 0.03447 to 0.03094, saving model to LSTM_basline.hdf5
Epoch 86/100
12637/12637 [==============================] - 120s 10ms/step - loss: 0.0271 - acc: 0.9996

Epoch 00086: loss improved from 0.03094 to 0.02711, saving model to LSTM_basline.hdf5
Epoch 87/100
12637/12637 [==============================] - 115s 9ms/step - loss: 0.0263 - acc: 0.9

In [37]:
from random import randint
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
 
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    for _ in range(n_words):
    # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
    # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [69]:
#test 1
# load cleaned text sequences
in_filename = 'Glenn_Schorr_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
model = load_model('GRU_0.hdf5')
 
# load the tokenizer
tokenizer = plk.load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

but in your text you point out low rates and slow economic growth as headwinds for ficc you talked about

lower market volumes and volatility and equities i just curious when you look at the quarter we just had in ficc particularly is it more like you had a nice pickup like and you setting us up for keep calm like things have returned back to normal or is this


In [85]:
model.load_weights(filepath)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
result_history = model.fit(X, y, batch_size=200, epochs= 500, callbacks=callbacks_list)

Epoch 1/500
12004/12004 [==============================] - 18s 2ms/step - loss: 3.3733 - acc: 0.2468

Epoch 00001: loss did not improve from 3.25588
Epoch 2/500
12004/12004 [==============================] - 12s 1ms/step - loss: 3.2386 - acc: 0.2646

Epoch 00002: loss improved from 3.25588 to 3.23858, saving model to LSTM_basline.hdf5
Epoch 3/500
12004/12004 [==============================] - 12s 1ms/step - loss: 3.2111 - acc: 0.2687

Epoch 00003: loss improved from 3.23858 to 3.21107, saving model to LSTM_basline.hdf5
Epoch 4/500
12004/12004 [==============================] - 13s 1ms/step - loss: 3.1937 - acc: 0.2726

Epoch 00004: loss improved from 3.21107 to 3.19372, saving model to LSTM_basline.hdf5
Epoch 5/500
12004/12004 [==============================] - 13s 1ms/step - loss: 3.1613 - acc: 0.2755

Epoch 00005: loss improved from 3.19372 to 3.16126, saving model to LSTM_basline.hdf5
Epoch 6/500
12004/12004 [==============================] - 13s 1ms/step - loss: 3.1517 - acc: 0.279

12004/12004 [==============================] - 13s 1ms/step - loss: 2.6505 - acc: 0.3688

Epoch 00047: loss improved from 2.66119 to 2.65046, saving model to LSTM_basline.hdf5
Epoch 48/500
12004/12004 [==============================] - 14s 1ms/step - loss: 2.6528 - acc: 0.3655

Epoch 00048: loss did not improve from 2.65046
Epoch 49/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.6265 - acc: 0.3700

Epoch 00049: loss improved from 2.65046 to 2.62651, saving model to LSTM_basline.hdf5
Epoch 50/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.6029 - acc: 0.3795

Epoch 00050: loss improved from 2.62651 to 2.60288, saving model to LSTM_basline.hdf5
Epoch 51/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.6177 - acc: 0.3751

Epoch 00051: loss did not improve from 2.60288
Epoch 52/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.6046 - acc: 0.3744

Epoch 00052: loss did not improve from 2.60

12004/12004 [==============================] - 15s 1ms/step - loss: 2.2306 - acc: 0.4513

Epoch 00093: loss did not improve from 2.22497
Epoch 94/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.1885 - acc: 0.4607

Epoch 00094: loss improved from 2.22497 to 2.18847, saving model to LSTM_basline.hdf5
Epoch 95/500
12004/12004 [==============================] - 14s 1ms/step - loss: 2.1967 - acc: 0.4597

Epoch 00095: loss did not improve from 2.18847
Epoch 96/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.1658 - acc: 0.4667

Epoch 00096: loss improved from 2.18847 to 2.16585, saving model to LSTM_basline.hdf5
Epoch 97/500
12004/12004 [==============================] - 14s 1ms/step - loss: 2.1459 - acc: 0.4714

Epoch 00097: loss improved from 2.16585 to 2.14593, saving model to LSTM_basline.hdf5
Epoch 98/500
12004/12004 [==============================] - 14s 1ms/step - loss: 2.1837 - acc: 0.4608

Epoch 00098: loss did not improve from 2.14

12004/12004 [==============================] - 14s 1ms/step - loss: 1.3776 - acc: 0.6558

Epoch 00188: loss improved from 1.40204 to 1.37761, saving model to LSTM_basline.hdf5
Epoch 189/500
12004/12004 [==============================] - 14s 1ms/step - loss: 1.3983 - acc: 0.6526

Epoch 00189: loss did not improve from 1.37761
Epoch 190/500
12004/12004 [==============================] - 14s 1ms/step - loss: 1.4129 - acc: 0.6432

Epoch 00190: loss did not improve from 1.37761
Epoch 191/500
12004/12004 [==============================] - 14s 1ms/step - loss: 1.3716 - acc: 0.6526

Epoch 00191: loss improved from 1.37761 to 1.37159, saving model to LSTM_basline.hdf5
Epoch 192/500
12004/12004 [==============================] - 92s 8ms/step - loss: 1.3663 - acc: 0.6541

Epoch 00192: loss improved from 1.37159 to 1.36629, saving model to LSTM_basline.hdf5
Epoch 193/500
12004/12004 [==============================] - 13s 1ms/step - loss: 1.3432 - acc: 0.6625

Epoch 00193: loss improved from 1.3662

12004/12004 [==============================] - 14s 1ms/step - loss: 0.7616 - acc: 0.8085

Epoch 00285: loss did not improve from 0.69694
Epoch 286/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7461 - acc: 0.8152

Epoch 00286: loss did not improve from 0.69694
Epoch 287/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7726 - acc: 0.8076

Epoch 00287: loss did not improve from 0.69694
Epoch 288/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7708 - acc: 0.8017

Epoch 00288: loss did not improve from 0.69694
Epoch 289/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7291 - acc: 0.8201

Epoch 00289: loss did not improve from 0.69694
Epoch 290/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7545 - acc: 0.8155

Epoch 00290: loss did not improve from 0.69694
Epoch 291/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7521 - acc: 0.8103

Epoc

11800/12004 [============================>.] - ETA: 0s - loss: 0.5160 - acc: 0.8738

KeyboardInterrupt: 

In [93]:
# result_history.history["loss"]

In [87]:
model.save('model.h6')

In [73]:
#test 1
# load cleaned text sequences
in_filename = 'Glenn_Schorr_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
# model = load_model('model.h6')

model.load_weights(filepath)
 
# load the tokenizer
tokenizer = plk.load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print('Initial text:')
print(seed_text + '\n')
 
# generate new text
print('Generated text:')
generated = generate_seq(model, tokenizer, seq_length, seed_text, 30)
print(generated)

Initial text:
competition and what we should expect on the deposit beta if there are more rate hikes and so maybe bluntly

Generated text:
if we get a couple more hate rate hikes do you capture the majority of it even without as you said a rate story do you have to hike along


In [80]:
#test 2

# load the tokenizer
tokenizer = plk.load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print('Initial text:')
print(seed_text + '\n')
 
# generate new text
print('Generated text:')
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

Initial text:
and are you still on track in your mind for the overhead ratio goals i do want to overly focus

Generated text:
on the dollar amount okay so i wanted to ask an nii question and i saw the comments on ex global markets the net interest yield being up basis points deposit betas are good so the core business that we all focus on is good i do want to talk


## Further improvement 

### 1. Improve the sequence build up based on each question, or expand training data to all questions from same category
### 2. Tune parameter to see if the performace is better in generating questions (batch size, epoch time etc.)
### 3. Research on the question generation process including tones and use of words
### 4. Improve the dropout (0.2, 0.4 0.6 etc.) to LSTM input layer to prevent overfitting for the model and may generate a more natural language

Dropout is a technique where randomly selected neurons are ignored during training. They are “dropped-out” randomly. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass.
paper about dropout
http://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf


